##Limpeza de Dados

In [0]:
import pandas as pd
#!pip install -U pandasql
import pandasql

# 1.Carregar o seu arquivo OVNIS.csv em um dataframe;
df = pd.read_csv("OVNIS.csv")


estados_eua = pd.read_csv("states.csv",sep=";")

# 2.Remover registros que tenham valores vazios (None, Unknown, ...) para City, State e Shape;
city = """ SELECT * FROM df WHERE LOWER(City) != 'unknown' AND LOWER(City)!= 'none' 
AND LOWER(City) != '' ;"""
sql_city = pandasql.sqldf(city)

state = """ SELECT * FROM sql_city WHERE LOWER(State) != 'unknown' AND LOWER(State)!= 'none' 
AND LOWER(State) != '' ;"""
sql_state = pandasql.sqldf(state)

shape = """ SELECT * FROM sql_state WHERE LOWER(Shape) != 'unknown' AND LOWER(Shape)!= 'none' 
AND LOWER(Shape) != '' ;"""
sql_shape = pandasql.sqldf(shape)

#3.Manter somente os registros referentes aos 51 estados dos Estados Unidos 
states = """ SELECT sql_shape.* FROM sql_shape, estados_eua WHERE sql_shape.State = estados_eua.Abbreviation; """
sql_eua = pandasql.sqldf(states)

#4. Remover variáveis irrelevantes para a análise (Duration, Summary e Posted);
sql_eua.drop(columns=['Duration','Summary','Posted', 'Unnamed: 0'], axis= 1, inplace=True)

# 5.Manter somente os registros de Shapes mais populares (com mais de 1000 ocorrências);
count = """ SELECT Shape , count(Shape) as ocorrencias FROM sql_eua 
GROUP BY Shape HAVING ocorrencias > 1000; """
sql_count = pandasql.sqldf(count)

ocorrencias = """ SELECT sql_eua.* FROM sql_eua, sql_count WHERE sql_count.Shape = sql_eua.Shape;"""
sql_ocorrencias = pandasql.sqldf(ocorrencias)

# 6.Salvar o dataframe final em um arquivo CSV com o nome "df_OVNI_limpo".
sql_ocorrencias.to_csv("df_OVNI_limpo.csv")
